In [2]:
import sys
sys.path.append('../src')
import preprocessing
import data_augmentation_1D as data_aug
import optuna 
import tensorflow as tf
import keras 
from keras.optimizers import Adam
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd

2024-03-07 11:32:24.353655: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-07 11:32:24.353723: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-07 11:32:24.355207: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-07 11:32:24.364866: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-07 11:32:26.015873: W tensorflow/compiler/tf2

In [2]:
# Définir le chemin racine du dataset
dataset_root = "/mnt/c/Users/onerf/Documents/UMons/Master2/RHEA/src/db/"

# Charger les profils depuis les dossiers spécifiés dans le dataset
profiles  = preprocessing.sleepers_dataset_from_directory(dataset_root + "sleepers_db/")
profiles_test = preprocessing.sleepers_dataset_from_directory(dataset_root + "test/")

2024-03-07 11:27:13.795720: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 11:27:14.177622: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 11:27:14.177893: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 11:27:14.182782: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 11:27:14.182890: I external/local_xla/xla/stream_executor

In [3]:
CLASSES = 3
INPUT_SHAPE = (437,1)
EPOCHS = 1
N_TRAIN_EXAMPLES = 14208
N_VALID_EXAMPLES = 400


x, y = profiles.data, profiles.target
# Redimensionnement des données pour ajouter une dimension supplémentaire,
x = x.reshape(x.shape[0], x.shape[1], 1)
print(f'x shape : {x.shape}')

x_test, y_test = profiles_test.data, profiles_test.target
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

x, y = data_augmentation_1D.augment(x, y)


x shape : (2220, 437, 1)


In [4]:
train_ds = tf.data.Dataset.from_tensor_slices((x,y)).shuffle().batch(32).take(N_TRAIN_EXAMPLES)

# valid_ds = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
# valid_ds = valid_ds.shuffle(len(x_test)).batch(32).take(N_VALID_EXAMPLES)

2024-03-07 11:29:09.680214: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 993423360 exceeds 10% of free system memory.
2024-03-07 11:29:12.012532: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 993423360 exceeds 10% of free system memory.


TypeError: DatasetV2.shuffle() missing 1 required positional argument: 'buffer_size'

In [ ]:
def create_model(trial,n_layers,max_pooling =False, drop_out=False):
    # We optimize the numbers of layers, their units and weight decay parameter.
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=INPUT_SHAPE))
    model.add(
        tf.keras.layers.BatchNormalization()
        )
    for i in range(n_layers):
        num_hidden = trial.suggest_categorical("n_units_l{}".format(i+1), [16, 32, 64, 128])

        kernel_size = trial.suggest_int("kernel_size", 1, 3)

        model.add(
            tf.keras.layers.Conv1D(num_hidden,kernel_size=kernel_size,padding="SAME",activation="relu", kernel_initializer="he_normal")
            )
        model.add(
          tf.keras.layers.BatchNormalization()
        )
        if(max_pooling):
          max_pool = trial.suggest_int("max_pool", 2, 3)
          model.add(
              tf.keras.layers.MaxPool1D(pool_size=max_pool)
          )
        if(drop_out):
          drop_out = trial.suggest_float("drop_out", 0.1, 0.5, log=True)
          model.add(
              tf.keras.layers.Dropout(drop_out)
          )

          
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(CLASSES, activation='softmax'))
    return model

In [ ]:
def learn(model, dataset, mode="eval"):
    accuracy = tf.metrics.Accuracy("accuracy", dtype=tf.float32)

    for batch, (images, labels) in enumerate(dataset):
        with tf.GradientTape() as tape:
            logits = model(images, training=(mode == "train"))
            loss_value = tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)
            )
            if mode == "eval":
                accuracy(
                    tf.argmax(logits, axis=1, output_type=tf.int64), tf.cast(labels, tf.int64)
                )
            else:
                grads = tape.gradient(loss_value, model.variables)
                optimizer.apply_gradients(zip(grads, model.variables))

    if mode == "eval":
        return accuracy

In [ ]:
def objective(trial):
    max_pooling = np.random.choice([True, False])
    drop_out = np.random.choice([True, False])
    model = create_model(trial,2,max_pooling,drop_out)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
   
    # Training and validating cycle.
    with tf.device("/cpu:0"):
      for _ in range(EPOCHS):
        model.compile(optimizer=Adam(learning_rate=learning_rate),
                            loss='sparse_categorical_crossentropy',
                            metrics=['accuracy'])

    # Return last validation accuracy.
    return model.summary()

In [ ]:
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=1)